## Ingest Circuits.csv file

### reading csv file in spark 

In [0]:
display(dbutils.fs.mounts())

In [0]:
%fs
ls /mnt/prisha12/raw

In [0]:
circuits_dataframe = spark.read.csv('dbfs:/mnt/prisha12/raw/circuits.csv',header=True, inferSchema=True)
# this is slow the process. because it is going through data which takes more time to consume. it is best to define schedma instead of letting system define. 

In [0]:
type(circuits_dataframe)

In [0]:
display(circuits_dataframe)

In [0]:
circuits_dataframe.printSchema()

In [0]:
display(circuits_dataframe.describe())

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType   

In [0]:
circuit_schema = StructType([StructField("circuitId", IntegerType(), False),
                             StructField("circuitRef", StringType(), True),
                             StructField("name", StringType(), True),
                             StructField("location", StringType(), True),
                             StructField("country", StringType(), True),
                             StructField("lat", DoubleType(), True),
                             StructField("lng", DoubleType(), True),
                             StructField("alt", IntegerType(), True),
                             StructField("url", StringType(), True)])

In [0]:
circuits_dataframe = spark.read.csv('dbfs:/mnt/prisha12/raw/circuits.csv',header=True, schema=circuit_schema)

In [0]:
display(circuits_dataframe.printSchema())

In [0]:
display(circuits_dataframe)

In [0]:
display(circuits_dataframe.describe())

In [0]:
circuits_selected_df = circuits_dataframe.select("circuitId","circuitRef","location","country","lat","lng","alt","url")

In [0]:
display(circuits_selected_df)

In [0]:
##renaming field names
circuits_selected_df = circuits_selected_df.withColumnRenamed("circuitId","circuiti_d").withColumnRenamed("circuitRef","circuit_ref").withColumnRenamed("lat","latitute").withColumnRenamed("lng","longitude")

In [0]:
display(circuits_selected_df)

### adding ingestion date of data

In [0]:
from pyspark.sql.functions import current_timestamp,lit

circuits_selected_df = circuits_selected_df.withColumn("ingestion_date", current_timestamp())
display(circuits_selected_df)

#### Write data to datalake as parquet

In [0]:
circuits_selected_df.write.mode('append').parquet("/mnt/prisha12/processed/circuits.parquet")

In [0]:
%fs
ls /mnt/prisha12/processed

In [0]:
df = spark.read.parquet("/mnt/prisha12/processed/circuits.parquet")

In [0]:
display(df)